## Описание проекта

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.
Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком.
Постройте модель с предельно большим значением F1-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте F1-меру на тестовой выборке самостоятельно.
Дополнительно измеряйте AUC-ROC, сравнивайте её значение с F1-мерой.

## Описание данных

Признаки

    RowNumber — индекс строки в данных
    CustomerId — уникальный идентификатор клиента
    Surname — фамилия
    CreditScore — кредитный рейтинг
    Geography — страна проживания
    Gender — пол
    Age — возраст
    Tenure — сколько лет человек является клиентом банка
    Balance — баланс на счёте
    NumOfProducts — количество продуктов банка, используемых клиентом
    HasCrCard — наличие кредитной карты
    IsActiveMember — активность клиента
    EstimatedSalary — предполагаемая зарплата

Целевой признак

    Exited — факт ухода клиента

In [103]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

### Шаг 1. Загрузка и знакомство с данными

In [2]:
# загрузим и посмотрим на данные
data = pd.read_csv('https://code.s3.yandex.net/datasets/Churn.csv')
data.name = 'churn_clients'
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [4]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
RowNumber,10000.0,5.000500e+03,2886.895680,1.00,2500.75,5.000500e+03,7.500250e+03,10000.00
CustomerId,10000.0,1.569094e+07,71936.186123,15565701.00,15628528.25,1.569074e+07,1.575323e+07,15815690.00
CreditScore,10000.0,6.505288e+02,96.653299,350.00,584.00,6.520000e+02,7.180000e+02,850.00
Age,10000.0,3.892180e+01,10.487806,18.00,32.00,3.700000e+01,4.400000e+01,92.00
Tenure,9091.0,4.997690e+00,2.894723,0.00,2.00,5.000000e+00,7.000000e+00,10.00
Balance,10000.0,7.648589e+04,62397.405202,0.00,0.00,9.719854e+04,1.276442e+05,250898.09
NumOfProducts,10000.0,1.530200e+00,0.581654,1.00,1.00,1.000000e+00,2.000000e+00,4.00
HasCrCard,10000.0,7.055000e-01,0.455840,0.00,0.00,1.000000e+00,1.000000e+00,1.00
IsActiveMember,10000.0,5.151000e-01,0.499797,0.00,0.00,1.000000e+00,1.000000e+00,1.00
EstimatedSalary,10000.0,1.000902e+05,57510.492818,11.58,51002.11,1.001939e+05,1.493882e+05,199992.48


In [6]:
# проверим датасет на пропуски
def get_missing_values(data: pd.DataFrame) -> None:
    """
    Выводит данные о пропусках в колонках по датафрейму.
    Не изменяет данные внутри датафрейма.

    :param data: pd.DataFrame
    :return: None
    """
    # получаем имена колонок датафрейма
    columns = data.columns.to_list()
    data_len = len(data)
    # объявляем счетчик
    counter = -1
    print('='*60)
    # если есть пропуски в данных - выводим информацию о пропусках по колонкам
    if sum(data.isnull().sum()) > 0:
        print(f'Количество записей в датафрейме {data.name}: {data_len} \n')
        print(f'В датафрейме {data.name} имеются следующие пропуски:')
        for i in data.isnull().sum():
            counter += 1
            if i > 0:
                print(f'  - в колонке {columns[counter]}: {i} пропусков, это {i/data_len:0.2%} об общего объема данных')
    else:
        print(f'Отлично, в датафрейме {data.name} отсутствуют пропуски.')

# посмотрим на пропуски в данных
get_missing_values(data)

Количество записей в датафрейме churn_clients: 10000 

В датафрейме churn_clients имеются следующие пропуски:
  - в колонке Tenure: 909 пропусков, это 9.09% об общего объема данных


In [7]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [11]:
fig = px.bar(
    data.groupby('Exited').count().rename({'RowNumber': 'count'}, axis=1).rename({0: 'Пользователей осталось', 1: 'Пользователей ушло'})['count'],
    text_auto=True,
    title="Распределение покинувших и оставшихся в «Бета-Банке» пользователей"
)

fig.update_layout(
    xaxis_title="Статус",
    yaxis_title="Количество пользователей"
)

fig.show()

Имеем 909 пропусков в колонке Tenure, в которой содержится информация о том, сколько лет человек является клиентом банка,
а также дисбаланс классов, ушло почти в 4 раза меньше пользователей, чем осталось.
Также столбцы 'RowNumber', 'CustomerId', 'Surname' можем удалить, т.к. они не будут влиять на качество обучения модели.

### Шаг 2. Предобработка данных.

Удалим бесполезные для обучения столбцы.

In [12]:
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

Для заполнения пропусков проверим, коррелирует ли столбец Tenure с какими-либо другими данными, если да, то сгруппируем данные по коррелирующим столбцам и заполним пропуски более релевантно.

In [13]:
data.corr()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
CreditScore,1.000000,-0.003965,-0.000062,0.006268,0.012238,-0.005458,0.025651,-0.001384,-0.027094
Age,-0.003965,1.000000,-0.013134,0.028308,-0.030680,-0.011721,0.085472,-0.007201,0.285323
Tenure,-0.000062,-0.013134,1.000000,-0.007911,0.011979,0.027232,-0.032178,0.010520,-0.016761
Balance,0.006268,0.028308,-0.007911,1.000000,-0.304180,-0.014858,-0.010084,0.012797,0.118533
NumOfProducts,0.012238,-0.030680,0.011979,-0.304180,1.000000,0.003183,0.009612,0.014204,-0.047820
HasCrCard,-0.005458,-0.011721,0.027232,-0.014858,0.003183,1.000000,-0.011866,-0.009933,-0.007138
IsActiveMember,0.025651,0.085472,-0.032178,-0.010084,0.009612,-0.011866,1.000000,-0.011421,-0.156128
EstimatedSalary,-0.001384,-0.007201,0.010520,0.012797,0.014204,-0.009933,-0.011421,1.000000,0.012097
Exited,-0.027094,0.285323,-0.016761,0.118533,-0.047820,-0.007138,-0.156128,0.012097,1.000000


Явной корреляции нет.
Проверим, какие значение самые частотные

In [14]:
fig = px.histogram(
    data['Tenure'],
    title=f"Распределение значений в колонке Tenure"
)

fig.update_layout(
    xaxis_title="Значение Tenure",
    yaxis_title="Количество сэмплов"
)

fig.show()

In [15]:
data['Tenure'].mean()

4.997690023099769

Реже всего всего встречаются значение 0 и 10, а значения от 1 до 9 имеют почти одинаковую частотность, среднее значение при этом равно 5 лет.
Думаю заполнив пропуски средним значением есть риск получить более плохие результаты работы модели из-за плохой релевантности заполнения данных.
Заполним пропуски наиболее похожими клиентами, для этого сгруппируем клиентов по стране и возрасту. Думаю это даст наиболее релевантное заполнение пропусков.

In [16]:
data['Tenure'] = data['Tenure'].fillna(np.ceil(data.groupby(by=['Geography', 'Age'])['Tenure'].transform('mean')))
# проверим пропуски
data.isna().sum()

CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

 Закодируем категориальные данные методом one-hot-encoding

In [17]:
# кодируем данные в новый датафрейм
gender_ohe = pd.get_dummies(data["Gender"], drop_first=True)
country_ohe = pd.get_dummies(data["Geography"], drop_first=True)
# удалим закодированные колонки
data.drop(["Gender", "Geography"], axis=1, inplace=True)
# склеим закодированные колонки с основным датафреймом
data = pd.concat([data, gender_ohe, country_ohe], axis=1)
# проверим результат
data.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Male,Germany,Spain
0,619,42,2.0,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,0,0,1
2,502,42,8.0,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1.0,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2.0,125510.82,1,1,1,79084.10,0,0,0,1


In [23]:
features = data.drop('Exited', axis=1)
target = data['Exited']

features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.25)

display(features_train.shape)
display(target_train.shape)

(7500, 11)

(7500,)

Т.к. в датасете присутствует дисбаланс классов, сразу создадим два дополнительных датафрейма после обработки с помощью upsampling и downsempling

In [89]:
def up_sample(
        features: pd.DataFrame,
        target: pd.DataFrame,
        repeat: int=0,
        repeat_auto: bool=False,
        zeros: bool=True) -> tuple[pd.DataFrame, pd.DataFrame]:

    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    # автоматическое сэмплирование до равных размеров features и target
    if repeat_auto:
        if zeros:
            repeat_features = np.round(len(features_ones) / len(features_zeros)).astype('int')
            repeat_target = np.round(len(target_ones) / len(target_zeros)).astype('int')
            features_upsampled = pd.concat([features_ones] + [features_zeros] * repeat_features)
            target_upsampled = pd.concat([target_ones] + [target_zeros] * repeat_target)

            features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=12345)

            return features_upsampled, target_upsampled
        else:
            repeat_features = np.round(len(features_zeros) / len(features_ones)).astype('int')
            repeat_target = np.round(len(target_zeros) / len(target_ones)).astype('int')
            features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat_features)
            target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat_target)

            features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=12345)

            return features_upsampled, target_upsampled

    else:
        features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
        target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)

    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)

    return features_upsampled, target_upsampled


def down_sample(
        features: pd.DataFrame,
        target: pd.DataFrame,
        n_samples: int,
        zeros: bool=True) -> tuple[pd.DataFrame, pd.DataFrame]:

    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    if zeros:
        features_sample = features_zeros.sample(n=n_samples, random_state=12345)
        target_sample = target_zeros.sample(n=n_samples, random_state=12345)

    else:
        features_sample = features_ones.sample(n=n_samples, random_state=12345)
        target_sample = target_ones.sample(n=n_samples, random_state=12345)

    features_downsampled = pd.concat([features_sample] + [features_ones])
    target_downsampled = pd.concat([target_sample] + [target_ones])

    features_downsampled = shuffle(features_downsampled, random_state=12345)
    target_downsampled = shuffle(target_downsampled, random_state=12345)

    return features_downsampled, target_downsampled

In [92]:
upsampled_features, upsampled_targets = up_sample(features_train, target_train, repeat_auto=True, zeros=False)
downsampled_features, downsampled_targets = down_sample(features_train, target_train, n_samples=len(features_train[target_train==1]))

In [100]:
fig = px.histogram(
    upsampled_targets,
    title=f"Распределение значений в колонке Tenure"
)

fig.update_layout(
    xaxis_title="Значение Tenure",
    yaxis_title="Количество сэмплов"
)

fig.show()

In [101]:
fig = px.histogram(
    downsampled_targets,
    title=f"Распределение значений в колонке Tenure"
)

fig.update_layout(
    xaxis_title="Значение Tenure",
    yaxis_title="Количество сэмплов"
)

fig.show()

### Обучение моделей

Для начала обучим модели на тех данных, что есть

In [104]:
model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_test)
print("F1:", f1_score(target_test, predicted_valid))

F1: 0.09326424870466321


In [ ]:
params = {
    'max_depth': list(range(5, 40)),
    'n_estimators': list(range(1, 40)),
    'bootstrap': [True, False]
}

# воспользуемся GridSearchCV для поиска наилучших гиперпараметров
random_forest_model = GridSearchCV(RandomForestClassifier(random_state=10), params, cv=5, n_jobs=4, verbose=10)
random_forest_model.fit(features_train, target_train)
print(f'Параметры наилучшей модели: {random_forest_model.best_params_}')
print(f'Accuracy: {random_forest_model.best_score_}')

Fitting 5 folds for each of 2730 candidates, totalling 13650 fits


In [106]:
model = SVC(random_state=12345)
model.fit(features_train, target_train)
predicted_valid = model.predict(features_test)
print("F1:", f1_score(target_test, predicted_valid))

F1: 0.0


In [107]:
model = DecisionTreeClassifier(random_state=12345)
model.fit(features_train, target_train)
predicted_valid = model.predict(features_test)
print("F1:", f1_score(target_test, predicted_valid))

F1: 0.5054294175715697
